# Calculating trainsition states - part 1

Identifying the most compute effective approach to calculating transition states can be difficult. Let us guide you through the options in this short notebook. **Pre-requisite: Familiarity with ASE**

Breakdown of this tutorial:
I.) Creating a work case
II.) Methods with examples
III.) Common problems and how to solve them

First, let us create a toy model of Au FCC(111) surface and 2 Cu ad atoms. 

In [ ]:
from examples.data.model_gen import get_example_slab
slab_initial = get_example_slab(adsorbate=True, type="2Cu")

In the model above the two Cu atoms are situated on top of hcp adsorption sites. The final geometry will be with copper atoms placed in non-neighbouring fcc and hcp sites. The goal is to calculate the transition state between the two and an activation energy.

In [ ]:
slab_final = get_example_slab(adsorbate=True, type="2Cu")
hcp_position = [slab_final[1].x, (slab_final[15].y + slab_final[1].y)/2]
# place the copper atom in the position specified above
slab_final[19].x, slab_final[19].y = hcp_position[0], hcp_position[1]

Now before we proceed we need to make sure the structures are optimised. Forces convergence criterion is usually below 0.01 eV/Angstrom on each individual atom, but for the purposes of this tutorial a looser 0.05 value is used.

In [ ]:
from ase.optimize import FIRE
from ase.constraints import FixAtoms
# For the purpose of this tutorial a  constraint is set for all of the surface atoms
# Constraints used should always be specific to your system and applied following tests
freeze = FixAtoms([atom.index for atom in slab_initial if atom.tag > 0])
slab_initial.set_constraint(freeze)
slab_final.set_constraint(freeze)

opt = FIRE(slab_initial, trajectory="slab_initial.traj")
opt.run(fmax=0.05)
opt = FIRE(slab_final, trajectory="slab_final.traj")
opt.run(fmax=0.05)

# You can have a look at the model using the visualizer
from ase.visualize import view
#view(slab_initial)
#view(slab_final)

Now that we have the structures prepared, we need to look for an activation energy for transition between slab_initial and slab_final.

There are multiple ways one can approach this problem, but all will in the end help to establish highest energy geometry linking these two structures - the transition state.

    a.) NEB
    b.) MLNEB
    c.) AutoNEB
    d.) Surface diffusion energy barriers using ASE constraints - fixed plane
    e.) Surface diffusion energy barriers using ASE constraints - fixed bond lengths
    f.) Others not included in this tutorial - Dimer Method (and Improved Dimer Method), Growing String etc.

a.) Nudged Elastic Band - NEB

"The Nudged Elastic Band method is a technique for finding transition paths (and corresponding energy barriers) between given initial and final states. The method involves constructing a “chain” of “replicas” or “images” of the system and relaxing them in a certain way."

Detailed description of the NEB class as implemented in ASE, including useful publications:
https://wiki.fysik.dtu.dk/ase/ase/neb.html?highlight=neb#module-ase.neb
ASE take on this tutorial, exploring the functionality a bit more:
https://wiki.fysik.dtu.dk/ase/tutorials/neb/diffusion.html

The "classic" NEB is a bit of a brute force approach. It takes a lot of trial and error to get below convergence criteria and it is costly. All of the "images" mentioned above are optimised per iteration, so if one is interested in getting to know the Minimum Energy Path (MEP) better, often number of DFT calculations goes into thousands.

One of the problems associated with NEBs in general is that they rely heavily on the user-provided input structures. If they are not optimal due to complexity of the process, the MEP is elongated and the overall experiment becomes more expensive. There are ways of improving initial/final guesses, but not as straightforward with regular NEB.

To overcome the complexity of the energy landscape we need to focus solely on the highest energy transition state and ignore the lesser maxima, which can arise due to rotations etc. For this we use a *climbing image* calculation (see NEB wiki for more info).

In [ ]:
# a) NEB code in ASE
from ase.neb import NEB
from ase.calculators.emt import EMT
import copy

# Make a band consisting of 7 images (including initial and final)
# Very important - each image has to have a separate calculator!
# If one wants to use a single calculator, e.g. sockets with FHI-aims,
# the SingleCalculatorNEB class needs to be used in instead

images = [slab_initial]
images += [slab_initial.copy() for i in range(5)]
images += [slab_final]
neb = NEB(images, climb=True) # use climbing image calculation

# Interpolate linearly the potisions of the three middle images:
neb.interpolate()

# Set calculators:
for image in images[1:6]:
    image.set_calculator(EMT())

# Optimize:
optimizer = FIRE(neb,
                 trajectory='A2B.traj')
optimizer.run(fmax=0.05)

# Use ASE GUI to view the MEP
#view(images)


In [ ]:
# Now let's analyse the output, if dealing with one calculation it is sufficient to use
# ASE GUI and choose Tools --> NEB to see the plot

import matplotlib.pyplot as plt
from ase.neb import NEBTools

nebtools = NEBTools(images)

# Get the calculated barrier and the energy change of the reaction.
Ef, dE = nebtools.get_barrier()

# Get the actual maximum force at this point in the simulation.
max_force = nebtools.get_fmax()

# Create a figure like that coming from ASE-GUI.
fig = nebtools.plot_band() # TODO: Why is this not generated when only runs once in Jupyter Notebook?
fig.savefig('barrier-neb.png')

# TODO: neb(✓), mlneb(✓), AutoNEB(✓) linear scan(✓), Dimer Method(can use guess from linear_scan) Growing string?)

# TODO: discuss indices in input structures (CO2 as example, avoid rotation during neb

# TODO: discuss symmetry operations (where possible) to reduce path size

# TODO: discuss difficult energy landscape, optimisations of local minima, dissociation with fixed bond lengths